In [2]:
import { csvToJson } from './helpers/csvHelpers.ts'
import {CsvFields} from './helpers/csvTypes.ts'

const pathToBirthCsv = './sourceData/Birth_Register.csv'
const pathToDeathCsv = './sourceData/Death_Register.csv'
const pathToMarriageCsv = './sourceData/Marriage_Register.csv'
const pathToAdoptionCsv = './sourceData/Adoption_Register.csv'
const pathToDeedpollCsv = './sourceData/Deedpoll.csv'

const csvData: CsvFields = {
  birth: await csvToJson(pathToBirthCsv),
  death: await csvToJson(pathToDeathCsv),
  marriage: await csvToJson(pathToMarriageCsv),
  adoption: await csvToJson(pathToAdoptionCsv),
  deedpoll: await csvToJson(pathToDeedpollCsv),
} 

In [14]:
import {locationsMap} from './lookupMappings/locations.ts'

const locations = csvData.birth
  .flatMap((record) => [
    record.MOTHERS_ADDRESS,
    record.MOTHERS_BIRTHPLACE,
    record.INFORMANTS_ADDRESS,
    record.CHILDS_BIRTHPLACE,
    record.FATHERS_BIRTHPLACE,
    record.FATHERS_ADDRESS,
  ])
  .filter((x) => x)

// const uniqueLocations = Array.from(new Set(locations))
//   .sort()
   const uniqueLocations = locationsMap.map((location) => ({
    name: location.name,
    map: location.map,
    facilityCode: null,
    country: null,
    intlTown: null
  }))

Deno.writeFileSync(
  './lookupMappings/locationsRaw.json',
  new TextEncoder().encode(JSON.stringify(uniqueLocations, null, 2)),
)

const seededLocations = await csvToJson('../../ckopencrvs/src/data-seeding/locations/source/locations.csv')

Deno.writeFileSync(
  './lookupMappings/locationsSeeded.json',
  new TextEncoder().encode(JSON.stringify(seededLocations, null, 2)),
)


const facilities = await csvToJson('../../ckopencrvs/src/data-seeding/locations/source/health-facilities.csv')
const facs = facilities.map((facility: any) => ({
  name: facility.name,
  id: facility.id,
}))

Deno.writeFileSync(
  './lookupMappings/facilities.json',
  new TextEncoder().encode(JSON.stringify(facs, null, 2)),
)


In [4]:
const informantTypes = csvData.birth
  .map((record) => record.INFORMANTS_RELATIONSHIP)
  .filter((x) => x)

const uniqueInformantTypes = Array.from(new Set(informantTypes)).sort()

Deno.writeFileSync(
  './lookupMappings/informantTypesRaw.json',
  new TextEncoder().encode(JSON.stringify(uniqueInformantTypes, null, 2)),
)

In [6]:
const nationalities = csvData.birth
  .flatMap((record) => [
    record.FATHERS_NATIONALITY,
    record.MOTHERS_NATIONALITY])
    .filter((x) => x)

const uniqueNationalities = Array.from(new Set(nationalities)).sort()

Deno.writeFileSync(
  './lookupMappings/nationalitiesRaw.json',
  new TextEncoder().encode(JSON.stringify(uniqueNationalities, null, 2)),
)

const races = csvData.birth
  .flatMap((record) => [
    record.FATHERS_RACE,
    record.MOTHERS_RACE])
    .filter((x) => x)

const uniqueRaces = Array.from(new Set(races)).sort()

Deno.writeFileSync(
  './lookupMappings/racesRaw.json',
  new TextEncoder().encode(JSON.stringify(uniqueRaces, null, 2)),
)

In [7]:
const twins = csvData.birth
  .map((record) => record.CHILDS_TWIN)
  .filter((x) => x)
const uniqueTwins = Array.from(new Set(twins)).sort()

Deno.writeFileSync(
  './lookupMappings/twinsRaw.json',
  new TextEncoder().encode(JSON.stringify(uniqueTwins, null, 2)),
)